<a href="https://colab.research.google.com/github/jpiche0720/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# DIRECTIONS

italicized text# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [X] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [X] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [X] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


#Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# BEGIN CODE

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [277]:
pd.set_option('display.max_columns',50)

In [73]:
print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,...,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,...,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,...,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,...,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,...,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,...,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


# Subset Data 

- for single family dwellings 100k - 2m



In [74]:
df['BUILDING_CLASS_CATEGORY'].value_counts()[:5]

01 ONE FAMILY DWELLINGS            5061
02 TWO FAMILY DWELLINGS            4567
10 COOPS - ELEVATOR APARTMENTS     3471
13 CONDOS - ELEVATOR APARTMENTS    3339
03 THREE FAMILY DWELLINGS          1438
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [315]:
data = df[df['BUILDING_CLASS_CATEGORY']== '01 ONE FAMILY DWELLINGS']
print(data.shape)
data.head()

(5061, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,01/01/2019
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,01/01/2019
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019


In [316]:
# Subset data
data = data[data['SALE_PRICE']>= 100000]
data = data[data['SALE_PRICE']<= 2000000]

# Convert to datetime format 
data['SALE_DATE'] = pd.to_datetime(data['SALE_DATE'])

# Drop Na values 
data = data.dropna(axis=1)

In [ ]:
data.dtypes

In [317]:
# Turn Land Square Feet to Int
data['LAND_SQUARE_FEET'] = data['LAND_SQUARE_FEET'].str.replace(',','')
data['LAND_SQUARE_FEET'] = data['LAND_SQUARE_FEET'].astype(int)

In [319]:
# Drop High Cardinallity Columns


columns = data.columns.to_list()
li = []
for col in columns:
    num = data[col].nunique()
    if num == 1:
        li.append(col)
    if num > 2500:
        li.append(col)
    

data = data.drop(columns=li)





# Split into Train/Test sets

- Train on JAN - MAR
- Test on APR

In [320]:
train = data[data['SALE_DATE']< pd.to_datetime('04/01/2019')]
test = data[data['SALE_DATE']>= pd.to_datetime('04/01/2019')]

train.shape,test.shape

((2517, 15), (647, 15))

In [321]:
target = 'SALE_PRICE'
features = train.columns.drop(['SALE_PRICE','SALE_DATE'])


X_train = train[features]
y_train = train[target]
X_test = test[features]
y_true = test[target]

# Modeling
## Feature Selection
- Select K Best with score func=f_regression, k=12

## Encoding
- One-Hot-encoder from category_encoders

## Regression Models
- Least Squares Regression
- Ridge Regression
- Ridge Regression with Cross Validation

## Metrics
- Mean Abs Error
- R2 score
- Max Error




In [329]:
import category_encoders as ce
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import numpy as np

import matplotlib.pyplot as plt

In [340]:
X_train.dtypes

BOROUGH_1                            int64
BOROUGH_2                            int64
BOROUGH_3                            int64
BOROUGH_4                            int64
BOROUGH_5                            int64
                                     ...  
BUILDING_CLASS_AT_TIME_OF_SALE_7     int64
BUILDING_CLASS_AT_TIME_OF_SALE_8     int64
BUILDING_CLASS_AT_TIME_OF_SALE_9     int64
BUILDING_CLASS_AT_TIME_OF_SALE_10    int64
BUILDING_CLASS_AT_TIME_OF_SALE_11    int64
Length: 167, dtype: object

In [325]:
# Encode Categorical Columns and zip code
# Instantiate the object
encoder = ce.OneHotEncoder(cols=['BOROUGH','NEIGHBORHOOD','TAX_CLASS_AT_PRESENT',
                                 'BUILDING_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE',
                                 'ZIP_CODE'])

# Fit/Transform X TRAIN feature matrix and Transform X TEST feature matrix 
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [ ]:
X_train.shape,X_test.shape

# Least Squares Regression

In [341]:
# Find Best number of features 
maes = []
feats = []
for k in range(1,46):
    selector = SelectKBest(score_func=f_regression,k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
  
    model = LinearRegression()
    model.fit(X_train_selected,y_train)
    y_pred = model.predict(X_test_selected)  
    mae = mean_absolute_error(y_true,y_pred)
    feat = k
    maes.append(mae)
    feats.append(feat) 
   
# Dataframe of lists
lowest_error = pd.DataFrame(maes,feats,columns=['mae'])['mae'].reset_index().nsmallest(1,columns=['mae'])
lowest_error = lowest_error.reset_index(drop=True)
feat_nums = lowest_error.iloc[0,0]

# Select Features
selector = SelectKBest(score_func=f_regression,k=feat_nums)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)


# Fit Ridge Model
model = LinearRegression()
model.fit(X_train_selected,y_train)
y_pred = model.predict(X_test_selected)

# Evaluation Metrics
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
print(f'With a Basic Least Squares Regression Model, The Mean Absolute Error with {feat} Features = ${mae:.0f}')
print(f'The R2 score is {r2}')

# Feature Coefficients
colnames = X_train.columns
selected_mask = selector.get_support()
col_sel = colnames[selected_mask]
col_unsel = colnames[~selected_mask]

feats = col_sel.to_list()
coef = model.coef_


coefs = pd.DataFrame(coef,feats,columns=['coefs'])
coefs = coefs.sort_values(by='coefs',ascending=False)

coefs

With a Basic Least Squares Regression Model, The Mean Absolute Error with 45 Features = $150947
The R2 score is 0.47631077072172945


# Ridge Regression

In [342]:
# Find Best number of features 
maes = []
feats = []
for k in range(1,46):
    selector = SelectKBest(score_func=f_regression,k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
  
    model = Ridge(normalize=True)
    model.fit(X_train_selected,y_train)
    y_pred = model.predict(X_test_selected)  
    mae = mean_absolute_error(y_true,y_pred)
    feat = k
    maes.append(mae)
    feats.append(feat) 

# Dataframe of lists    
lowest_error = pd.DataFrame(maes,feats,columns=['mae'])['mae'].reset_index().nsmallest(1,columns=['mae'])
lowest_error = lowest_error.reset_index(drop=True)
feat_nums = lowest_error.iloc[0,0]

# Select Features
selector = SelectKBest(score_func=f_regression,k=feat_nums)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)


# Fit Ridge Model
model = Ridge(normalize=True)
model.fit(X_train_selected,y_train)
y_pred = model.predict(X_test_selected)

# Evaluation Metrics
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
print(f'With a Basic Ridge Regression Model, The Mean Absolute Error with {feat} Features = ${mae:.0f}')
print(f'The R2 score is {r2}')

# Feature Coefficients
colnames = X_train.columns
selected_mask = selector.get_support()
col_sel = colnames[selected_mask]
col_unsel = colnames[~selected_mask]

feats = col_sel.to_list()
coef = model.coef_

coefs = pd.DataFrame(coef,feats,columns=['coefs'])

coefs

With a Basic Ridge Regression Model, The Mean Absolute Error with 45 Features = $163209
The R2 score is 0.38226854106182473


,coefs
BOROUGH_1,55075.324968
BOROUGH_3,-70375.284099
BOROUGH_4,-47864.352994
NEIGHBORHOOD_1,-81932.862607
NEIGHBORHOOD_2,64050.440147
NEIGHBORHOOD_4,147296.097679
NEIGHBORHOOD_5,130259.608900
TAX_CLASS_AT_PRESENT_1,2741.233817
BUILDING_CLASS_AT_PRESENT_3,-9295.306173
BUILDING_CLASS_AT_PRESENT_6,115161.758309


# RIDGECV

In [346]:
# Find Best number of features 
maes = []
feats = []
for k in range(1,46):
    selector = SelectKBest(score_func=f_regression,k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
  
    model = RidgeCV(normalize=True)
    model.fit(X_train_selected,y_train)
    y_pred = model.predict(X_test_selected)  
    mae = mean_absolute_error(y_true,y_pred)
    feat = k
    maes.append(mae)
    feats.append(feat) 
  
# Dataframe of lists
lowest_error = pd.DataFrame(maes,feats,columns=['mae'])['mae'].reset_index().nsmallest(1,columns=['mae'])
lowest_error = lowest_error.reset_index(drop=True)
feat_nums = lowest_error.iloc[0,0]

# Select Features
selector = SelectKBest(score_func=f_regression,k=feat_nums)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)


# Fit Ridge Model
model = RidgeCV(normalize=True)
model.fit(X_train_selected,y_train)
y_pred = model.predict(X_test_selected)

# Evaluation Metrics
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
print(f'With a Basic RidgeCV Regression Model, The Mean Absolute Error with {feat} Features = ${mae:.0f}')
print(f'The R2 score is {r2}')

# Feature Coefficients
colnames = X_train.columns
selected_mask = selector.get_support()
col_sel = colnames[selected_mask]
col_unsel = colnames[~selected_mask]

feats = col_sel.to_list()
coef = model.coef_

coefs = pd.DataFrame(coef,feats,columns=['coefs'])

coefs

With a Basic RidgeCV Regression Model, The Mean Absolute Error with 45 Features = $151351
The R2 score is 0.46978571479153874


,coefs
BOROUGH_1,4.953026e+04
BOROUGH_3,-1.366703e+05
BOROUGH_4,-9.812178e+04
NEIGHBORHOOD_1,-1.057510e+05
NEIGHBORHOOD_2,6.555141e+04
NEIGHBORHOOD_4,1.665522e+05
NEIGHBORHOOD_5,2.134567e+05
BUILDING_CLASS_AT_PRESENT_3,-6.000062e+02
BUILDING_CLASS_AT_PRESENT_6,9.366549e+04
BUILDING_CLASS_AT_PRESENT_7,4.536805e+04
